In [1]:
#download the dataset
import kagglehub
path = kagglehub.dataset_download("ntcuong2103/crohme2019")
path

'/kaggle/input/crohme2019'

In [ ]:
# authored by me for parsing and rendering InkML handwriting data

import os
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
from glob import glob

# parse an InkML file and optionally save the rendered trace image
def parse_and_render_inkml(file_path, save_path=None):
    """
    Parses an InkML (.inkml) file, extracts stroke data (traces),
    renders it as a handwriting image using matplotlib,
    and optionally saves the image to a file.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()

    traces = []
    # extract all <trace> elements
    for trace in root.findall(".//{http://www.w3.org/2003/InkML}trace"):
        raw_points = trace.text.strip().split(',')
        stroke = []
        for point in raw_points:
            coords = point.strip().split()
            if len(coords) == 2:
                stroke.append([float(coords[0]), float(coords[1])])
        if stroke:
            traces.append(np.array(stroke))

    # create a figure and draw the strokes
    fig, ax = plt.subplots()
    for stroke in traces:
        ax.plot(stroke[:, 0], -stroke[:, 1], linewidth=2)  # flip y-axis

    ax.axis('off')
    ax.set_aspect('equal')

    # save the figure if path provided, otherwise show it
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
        plt.close(fig)
    else:
        plt.show()

# explore folder contents and count files with optional extension
def explore_directory(dir_path, extension=None):
    print(f"Exploring directory: {dir_path}\n")
    if not os.path.exists(dir_path):
        print("Path does not exist.")
        return

    subdirs = [d for d in os.listdir(dir_path) if os.path.isdir(os.path.join(dir_path, d))]
    files = [f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))]

    print(f"Subfolders: {len(subdirs)}")
    for d in subdirs:
        print("  └──", d)

    if extension:
        matching = [f for f in files if f.endswith(extension)]
        print(f"\n .{extension} files: {len(matching)}")
    else:
        print(f"\n Total files: {len(files)}")

# check contents of all relevant data folders
explore_directory("/kaggle/input/crohme2019/crohme2019/crohme2019/test/", ".inkml")
explore_directory("/kaggle/input/crohme2019/crohme2019/crohme2019/train/", ".inkml")
explore_directory("/kaggle/input/crohme2019/crohme2019/crohme2019/valid/", ".inkml")


Exploring directory: /kaggle/input/crohme2019/crohme2019/crohme2019/test/

Subfolders: 0

 ..inkml files: 1199
Exploring directory: /kaggle/input/crohme2019/crohme2019/crohme2019/train/

Subfolders: 0

 ..inkml files: 8901
Exploring directory: /kaggle/input/crohme2019/crohme2019/crohme2019/valid/

Subfolders: 0

 ..inkml files: 986


In [ ]:
# parsing label file and saving structured labels to CSV
import pandas as pd

# this is the path to the ground-truth label file
label_file_path = "/kaggle/input/crohme2019/crohme2019_train.txt"

# list to hold parsed (file_path, label_text) pairs
label_entries = []

# read each line from the label file and split into components
with open(label_file_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:  # skip empty lines
            parts = line.split(maxsplit=1)  # split into path and label
            if len(parts) == 2:
                label_entries.append(parts)

# convert parsed data into a DataFrame
df = pd.DataFrame(label_entries, columns=["file_path", "label"])

# extract just the file name from the path (for easier matching later)
df["file_name"] = df["file_path"].apply(lambda x: x.split("/")[-1])

# tokenize label string into list of tokens
df["tokens"] = df["label"].str.split()

# export the DataFrame to a CSV file for later use
df.to_csv("/kaggle/working/train_labels.csv", index=False)
print(" Label data saved to train_labels.csv")

Label data saved to train_labels.csv


In [ ]:
# parsing label file and saving structured labels to CSV
import pandas as pd

# this is the path to the ground-truth label file for test set
label_file_path = "/content/crohme2019_test.txt"

# list to hold parsed (file_path, label_text) pairs
label_entries = []

# read each line from the label file and split into components
with open(label_file_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:  # skip empty lines
            parts = line.split(maxsplit=1)  # split into path and label
            if len(parts) == 2:
                label_entries.append(parts)

# convert parsed data into a DataFrame
df = pd.DataFrame(label_entries, columns=["file_path", "label"])

# extract just the file name from the path (for easier matching later)
df["file_name"] = df["file_path"].apply(lambda x: x.split("/")[-1])

# tokenize label string into list of tokens
df["tokens"] = df["label"].str.split()

# export the DataFrame to a CSV file for later use
df.to_csv("test_labels.csv", index=False)
print(" Label data saved to test_labels.csv")

Label data saved to test_labels.csv


In [ ]:
# parsing label file and saving structured labels to CSV
import pandas as pd

# this is the path to the ground-truth label file for validation set
label_file_path = "/content/crohme2019_valid.txt"

# list to hold parsed (file_path, label_text) pairs
label_entries = []

# read each line from the label file and split into components
with open(label_file_path, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if line:  # skip empty lines
            parts = line.split(maxsplit=1)  # split into path and label
            if len(parts) == 2:
                label_entries.append(parts)

# convert parsed data into a DataFrame
df = pd.DataFrame(label_entries, columns=["file_path", "label"])

# extract just the file name from the path (for easier matching later)
df["file_name"] = df["file_path"].apply(lambda x: x.split("/")[-1])

# tokenize label string into list of tokens
df["tokens"] = df["label"].str.split()

# export the DataFrame to a CSV file for later use
df.to_csv("valid_labels.csv", index=False)
print(" Label data saved to valid_labels.csv")

Label data saved to valid_labels.csv


In [ ]:
import pandas as pd
from collections import Counter
import json

# load the CSV file containing training labels and their tokenized math expressions
train_df = pd.read_csv("/content/train_labels.csv")

# in case the token column doesn't exist, generate it from the label string
if "tokens" not in train_df.columns:
    train_df["tokens"] = train_df["label"].astype(str).str.split()

# flatten all tokens across all samples into a single list
# this allows us to find all unique tokens used in the training data
all_tokens = [token for tokens in train_df["tokens"] for token in tokens if isinstance(token, str)]

# count frequency of each token (can be useful for analysis)
token_counts = Counter(all_tokens)

# define special tokens used by the model (padding, start/end, unknown)
special_tokens = ["<PAD>", "<SOS>", "<EOS>", "<UNK>"]

# collect all unique tokens from the dataset and sort them alphabetically
unique_tokens = sorted(set(all_tokens))

# construct the final vocabulary list: special tokens first, then all unique tokens
vocab = special_tokens + unique_tokens

# create a dictionary that maps each token to a unique integer ID
token_to_id = {token: idx for idx, token in enumerate(vocab)}

# create the reverse mapping: ID back to token (used during decoding)
id_to_token = {idx: token for token, idx in token_to_id.items()}

# export the token-to-ID mapping to a JSON file for later use during training
with open("token_to_id.json", "w", encoding="utf-8") as f:
    json.dump(token_to_id, f, indent=2, ensure_ascii=False)

# also export the reverse mapping (optional but useful for evaluation/inference)
with open("id_to_token.json", "w", encoding="utf-8") as f:
    json.dump(id_to_token, f, indent=2, ensure_ascii=False)

# confirmation message
print(f" Vocabulary saved! Total tokens: {len(vocab)}")


 Vocabulary saved! Total tokens: 77


In [3]:
import pandas as pd
import json
import os

# load token-to-id mapping from previously generated vocab
with open("token_to_id.json", "r", encoding="utf-8") as f:
    token_to_id = json.load(f)

# helper function to encode tokens using vocab (with <UNK> fallback)
def encode_tokens(tokens, vocab):
    return [vocab.get(token, vocab["<UNK>"]) for token in tokens]

# process label CSV and add encoded sequences
def encode_label_file(csv_path):
    if not os.path.exists(csv_path):
        print(f"File not found: {csv_path}")
        return

    print(f"Processing: {csv_path}")
    df = pd.read_csv(csv_path)

    # tokenize if necessary
    if "tokens" not in df.columns:
        df["tokens"] = df["label"].astype(str).str.split()

    # encode tokens to integer IDs
    df["label_ids"] = df["tokens"].apply(lambda tokens: encode_tokens(tokens, token_to_id))
    df["label_ids_with_sos_eos"] = df["label_ids"].apply(
        lambda ids: [token_to_id["<SOS>"]] + ids + [token_to_id["<EOS>"]]
    )

    # save back to same CSV
    df.to_csv(csv_path, index=False)
    print(f"Saved encoded labels to: {csv_path}\n")

# paths to all 3 label files
label_files = [
    "train_labels.csv",
    "valid_labels.csv",
    "test_labels.csv"
]

# apply encoding to all label files
for file_path in label_files:
    encode_label_file(file_path)


Processing: train_labels.csv
Saved encoded labels to: train_labels.csv

Processing: valid_labels.csv
Saved encoded labels to: valid_labels.csv

Processing: test_labels.csv
Saved encoded labels to: test_labels.csv



In [8]:
import os
import cv2
import numpy as np
from xml.etree import ElementTree as ET
from glob import glob
from tqdm import tqdm
import random

# Paths
INKML_FOLDER = "/kaggle/input/crohme2019/crohme2019/crohme2019/train/"
OUTPUT_FOLDER = "train images/"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
IMG_SIZE = (128, 128)

# Parse InkML file and extract list of valid strokes
def parse_inkml_traces(file_path):
    traces = []
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        for trace in root.findall(".//{http://www.w3.org/2003/InkML}trace"):
            raw = trace.text.strip().split(',')
            stroke = []
            for pt in raw:
                coords = pt.strip().split()
                if len(coords) == 2:
                    stroke.append([float(coords[0]), float(coords[1])])
            if len(stroke) >= 2:
                traces.append(np.array(stroke))
    except Exception as e:
        raise ValueError(f"Could not parse {file_path}: {e}")
    return traces

# Render strokes onto canvas and preprocess the image
def render_image(traces, augment=True):
    canvas = np.ones((600, 600), dtype=np.uint8) * 255  # white background

    for stroke in traces:
        for i in range(len(stroke) - 1):
            pt1 = tuple(map(int, stroke[i]))
            pt2 = tuple(map(int, stroke[i + 1]))
            cv2.line(canvas, pt1, pt2, color=0, thickness=2)

    # resize whole canvas directly (no cropping)
    resized = cv2.resize(canvas, IMG_SIZE, interpolation=cv2.INTER_AREA)

    if augment:
        angle = random.uniform(-5, 5)
        M = cv2.getRotationMatrix2D((IMG_SIZE[1] // 2, IMG_SIZE[0] // 2), angle, 1)
        resized = cv2.warpAffine(resized, M, IMG_SIZE, borderValue=255)

    return resized.astype(np.float32) / 255.0  # normalize to [0, 1]

# Process all .inkml files and render them to PNG
inkml_files = glob(os.path.join(INKML_FOLDER, "*.inkml"))
print(f"Found {len(inkml_files)} .inkml files to process.")

failed_files = []

for inkml_path in tqdm(inkml_files):
    file_name = os.path.splitext(os.path.basename(inkml_path))[0]
    output_path = os.path.join(OUTPUT_FOLDER, file_name + ".png")

    try:
        traces = parse_inkml_traces(inkml_path)
        image = render_image(traces, augment=True)
        image_uint8 = (image * 255).astype(np.uint8)
        cv2.imwrite(output_path, image_uint8)

    except Exception as e:
        failed_files.append(file_name)
        print(f"Failed to render {file_name}: {e}")

print(f"\n Total files that failed to render: {len(failed_files)}")
if failed_files:
    print("Example failed files:", failed_files[:5])


Found 8901 .inkml files to process.


 48%|████▊     | 4284/8901 [00:19<00:20, 230.19it/s]

Failed to render MfrDB0104: Could not parse /kaggle/input/crohme2019/crohme2019/crohme2019/train/MfrDB0104.inkml: not well-formed (invalid token): line 15, column 23


100%|██████████| 8901/8901 [00:42<00:00, 207.77it/s]


 Total files that failed to render: 1
Example failed files: ['MfrDB0104']


In [9]:
import tensorflow as tf
import pandas as pd
import json
import numpy as np
import os

# path to the CSV file that contains label info
CSV_PATH = "/content/train_labels.csv"

# path to folder where images (PNG) are stored
IMAGE_FOLDER = "/content/train_images/"

# path to vocab file for reading the PAD token ID
VOCAB_FILE = "/content/token_to_id.json"

# final input size expected by the model
IMG_SIZE = (128, 128)

# load the label CSV file
df = pd.read_csv(CSV_PATH)

# load the vocab to get the PAD token index
with open(VOCAB_FILE, "r", encoding="utf-8") as f:
    token_to_id = json.load(f)
pad_token_id = token_to_id["<PAD>"]

# make sure label column is present
if "label_ids_with_sos_eos" not in df.columns:
    raise ValueError("Missing label_ids_with_sos_eos column. Please run encoding first.")

# convert stringified list in CSV to actual list of integers
def str_to_list(s):
    return list(map(int, s.strip("[]").split(",")))

df["label_ids_with_sos_eos"] = df["label_ids_with_sos_eos"].apply(str_to_list)

# determine max sequence length for padding
max_seq_len = max(df["label_ids_with_sos_eos"].apply(len))

# build full paths to images and pad all label sequences
image_paths = [os.path.join(IMAGE_FOLDER, fname.replace(".inkml", ".png")) for fname in df["file_name"]]
label_seqs = [seq + [pad_token_id] * (max_seq_len - len(seq)) for seq in df["label_ids_with_sos_eos"]]

# create a raw TensorFlow dataset from file paths and labels
dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_seqs))

# define a function that loads and preprocesses each image
def process_sample(image_path, label_seq):
    image = tf.io.read_file(image_path)                      # read file
    image = tf.image.decode_png(image, channels=1)           # decode grayscale PNG
    image = tf.image.convert_image_dtype(image, tf.float32)  # normalize to [0, 1]
    image = tf.image.resize(image, IMG_SIZE)                 # ensure fixed size
    return image, tf.cast(label_seq, tf.int32)

# apply preprocessing to every item in the dataset
dataset = dataset.map(process_sample, num_parallel_calls=tf.data.AUTOTUNE)

# shuffle, batch, and prefetch for training
BATCH_SIZE = 32
train_dataset = dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
